In [1]:
import os
import sqlite3
import sys

sys.path.append("..")

import nivapy3 as nivapy
import numpy as np
import pandas as pd
import utils

# Tiltaksovervakingen: opsjon for kvalitetskontroll av analysedata
# Eurofins 2025 Q1

## Notebook 1: Initial exploration and data cleaning

## 1. Initial data check

**Note:** This file uses an updated export from Vannmiljø reflecting corrections made to two station codes in August 2021 - see e-mail from Kjetil received 27.05.2021 at 17:15 for details.

In [2]:
# Choose dataset to process
lab = "Eurofins"
year = 2025
qtr = 1
version = 1

# Define reference datasets
ref_stn_xls = r"../../data/all_stations_2025-11-13.xlsx"
ref_vm_xls = r"../../data/vannmiljo_export_2012-2024_2025-11-13.xlsx"
ref_st_yr, ref_end_yr = 2012, 2024

## 2. Create SQLite database to store results

Using a database will provide basic checks on data integrity and consistency. For this project, three tables will be sufficient:

 * Station locations and metadata
 * Parameters and units used by VestfoldLAB, Eurofins and Vannmiljø, and conversion factors between these
 * Water chemistry data
 
The code below creates a basic database structure, which will be populated later.

In [3]:
# Create database
fold_path = f"../../output/{lab.lower()}_{year}_q{qtr}_v{version}"
if not os.path.exists(fold_path):
    os.makedirs(fold_path)

db_path = os.path.join(fold_path, "kalk_data.db")
if os.path.exists(db_path):
    os.remove(db_path)
eng = sqlite3.connect(db_path, detect_types=sqlite3.PARSE_DECLTYPES)

# Turn off journal mode for performance
eng.execute("PRAGMA synchronous = OFF")
eng.execute("PRAGMA journal_mode = OFF")
eng.execute("PRAGMA foreign_keys = ON")

# Create stations table
sql = (
    "CREATE TABLE stations "
    "( "
    "  fylke text NOT NULL, "
    "  vassdrag text NOT NULL, "
    "  station_name text NOT NULL, "
    "  station_number text, "
    "  vannmiljo_code text NOT NULL, "
    "  vannmiljo_name text, "
    "  utm_east real NOT NULL, "
    "  utm_north real NOT NULL, "
    "  utm_zone integer NOT NULL, "
    "  lon real NOT NULL, "
    "  lat real NOT NULL, "
    "  liming_status text NOT NULL, "
    "  comment text, "
    "  PRIMARY KEY (vannmiljo_code) "
    ")"
)
eng.execute(sql)

# Create parameters table
sql = (
    "CREATE TABLE parameters_units "
    "( "
    "  vannmiljo_name text NOT NULL UNIQUE, "
    "  vannmiljo_id text NOT NULL UNIQUE, "
    "  vannmiljo_unit text NOT NULL, "
    "  vestfoldlab_name text NOT NULL UNIQUE, "
    "  vestfoldlab_unit text NOT NULL, "
    "  vestfoldlab_to_vm_conv_fac real NOT NULL, "
    "  eurofins_name text NOT NULL UNIQUE, "
    "  eurofins_unit text NOT NULL, "
    "  eurofins_to_vm_conv_fac real NOT NULL, "
    "  min real NOT NULL, "
    "  max real NOT NULL, "
    "  PRIMARY KEY (vannmiljo_id) "
    ")"
)
eng.execute(sql)

# Create chemistry table
sql = (
    "CREATE TABLE water_chemistry "
    "( "
    "  vannmiljo_code text NOT NULL, "
    "  sample_date datetime NOT NULL, "
    "  lab text NOT NULL, "
    "  period text NOT NULL, "
    "  depth1 real, "
    "  depth2 real, "
    "  parameter text NOT NULL, "
    "  flag text, "
    "  value real NOT NULL, "
    "  unit text NOT NULL, "
    "  PRIMARY KEY (vannmiljo_code, sample_date, depth1, depth2, parameter), "
    "  CONSTRAINT vannmiljo_code_fkey FOREIGN KEY (vannmiljo_code) "
    "      REFERENCES stations (vannmiljo_code) "
    "      ON UPDATE NO ACTION ON DELETE NO ACTION, "
    "  CONSTRAINT parameter_fkey FOREIGN KEY (parameter) "
    "      REFERENCES parameters_units (vannmiljo_id) "
    "      ON UPDATE NO ACTION ON DELETE NO ACTION "
    ")"
)
eng.execute(sql)

## 3. Explore station data

Station details are a tidied version of Øyvind's original file here:

    K:\Prosjekter\langtransporterte forurensninger\Kalk Tiltaksovervåking\12 KS vannkjemi\Vannlokaliteter koordinater_kun aktive stasj 2020.xlsx
    
Note that corrections (e.g. adjusted station co-ordinates) have been made to the tidied file, but not the original on `K:`. **The version in this repository should therefore been used as the "master" copy**.

In [4]:
# Read station data
stn_df = pd.read_excel(ref_stn_xls, sheet_name="data")
stn_df = nivapy.spatial.utm_to_wgs84_dd(stn_df)

print("The following stations are missing spatial co-ordinates:")
stn_df.query("lat != lat")

The following stations are missing spatial co-ordinates:


,fylke,vassdrag,station_name,station_number,vannmiljo_code,vannmiljo_name,utm_east,utm_north,utm_zone,liming_status,comment,lat,lon


In [5]:
print("The following stations do not have a code in Vannmiljø:")
stn_df.query("vannmiljo_code != vannmiljo_code")

The following stations do not have a code in Vannmiljø:


,fylke,vassdrag,station_name,station_number,vannmiljo_code,vannmiljo_name,utm_east,utm_north,utm_zone,liming_status,comment,lat,lon


In [6]:
# Map
stn_map = nivapy.spatial.quickmap(
    stn_df.dropna(subset=["lat"]),
    lat_col="lat",
    lon_col="lon",
    popup="station_name",
    cluster=True,
    kartverket=True,
    aerial_imagery=True,
)

stn_map.save("../../pages/stn_map.html")

stn_map

In [7]:
# Add to database
stn_df.dropna(subset=["vannmiljo_code", "lat"], inplace=True)
stn_df.to_sql(name="stations", con=eng, if_exists="append", index=False)

224

## 4. Parameters and units of interest

The file `../../data/parameter_unit_mapping.xlsx` provides a lookup between parameter names & units used by the labs and those in Vannmiljø. It also contains plausible ranges (using Vannmiljø units) for each parameter. These ranges have been chosen by using the values already in Vannmiljø as a reference. However, it looks as though some of the data in Vannmiljø might also be spurious, so it would be good to refine these ranges based on domain knowledge, if possible.

**Note:** Concentrations reported as *exactly* zero are likely to be errors, because most (all?) lab methods should report an LOQ instead.

In [8]:
# Read parameter mappings
par_df = utils.get_par_unit_mappings()

# Add to database
par_df.to_sql(name="parameters_units", con=eng, if_exists="append", index=False)

par_df

,vannmiljo_name,vannmiljo_id,vannmiljo_unit,vestfoldlab_name,vestfoldlab_unit,vestfoldlab_to_vm_conv_fac,eurofins_name,eurofins_unit,eurofins_to_vm_conv_fac,min,max
0,Temperatur,TEMP,°C,Temp,°C,1.000000,Temp,°C,1.000000,-10,30
1,pH,PH,<ubenevnt>,pH,enh,1.000000,pH,enh,1.000000,1,10
2,Konduktivitet,KOND,mS/m,Kond,mS/m,1.000000,Kond,ms/m,1.000000,0,100
3,Total alkalitet,ALK,mmol/l,Alk,mmol/l,1.000000,Alk,mmol/l,1.000000,0,2
4,Totalfosfor,P-TOT,µg/l P,Tot-P,µg/l,1.000000,Tot-P,µg/l,1.000000,0,500
5,Totalnitrogen,N-TOT,µg/l N,Tot-N,µg/l,1.000000,Tot-N,µg/l,1.000000,0,4000
6,Nitrat,N-NO3,µg/l N,NO3,µg/l,1.000000,NO3,µg/l,1.000000,0,2000
7,Totalt organisk karbon (TOC),TOC,mg/l C,TOC,mg/l,1.000000,TOC,mg/l,1.000000,0,100
8,Reaktivt aluminium,RAL,µg/l Al,RAl,µg/l,1.000000,RAl,µg/l,1.000000,0,500
9,Ikke-labilt aluminium,ILAL,µg/l Al,ILAl,µg/l,1.000000,ILAl,µg/l,1.000000,0,500


## 5. Historic data from Vannmiljø

The Vannmiljø dataset is large and reading from Excel is slow; the code below takes a couple of minutes to run.

Note from the output below that **there are more than 1600 "duplicated" samples in the Vannmiljø dataset** i.e. where the station code, sample date, sample depth, lab and parameter name are all the same, but a different value is reported. It would be helpful to know why these duplicates were collected e.g. are these reanalysis values, where only one of the duplicates should be used, or are they genuine (in which case should they be averaged or kept separate?). **For the moment, I will ignore these values**.

In [9]:
# Read historic data from Vannmiljø
his_df = utils.read_historic_data(
    ref_vm_xls,
    st_yr=ref_st_yr,
    end_yr=ref_end_yr,
)

# Tidy lab names for clarity
his_df["lab"] = his_df["lab"].replace(
    {
        "NIVA": "NIVA (historic)",
        "VestfoldLAB AS": "VestfoldLAB (historic)",
        "Eurofins Environment Testing Norway AS (Moss)": "Eurofins (historic)",
    },
)

# Add label for data period
his_df["period"] = "historic"

# Print summary
n_stns = len(his_df["vannmiljo_code"].unique())
print(f"The number of unique stations with data is: {n_stns}.\n")

# Handle duplicates
his_dup_csv = r"../../output/vannmiljo_historic/vannmiljo_duplicates.csv"
his_df = utils.handle_duplicates(his_df, his_dup_csv, action="drop")

his_df.head()

The number of unique stations with data is: 223.


There are 5630 duplicated records (same station_code-date-depth-parameter, but different value).
These will be dropped.



,vannmiljo_code,sample_date,lab,depth1,depth2,par_unit,flag,value,period
0,027-28435,2012-01-02,NIVA (historic),0.0,0.0,ALK_mmol/l,=,0.060,historic
1,027-28435,2012-02-06,NIVA (historic),0.0,0.0,ALK_mmol/l,=,0.061,historic
2,027-28435,2012-03-05,NIVA (historic),0.0,0.0,ALK_mmol/l,=,0.050,historic
3,027-28435,2012-04-02,NIVA (historic),0.0,0.0,ALK_mmol/l,=,0.058,historic
4,027-28435,2012-05-17,NIVA (historic),0.0,0.0,ALK_mmol/l,=,0.077,historic


## 6. New data from labs

The code below reads the Excel template provided by Eurofins and reformats it to the same structure (parameter names, units etc.) as the data in Vannmiljø.

In [10]:
# Read new data
new_df = utils.read_data_template_to_wide(
    f"../../data/{lab.lower()}_data_{year}_q{qtr}_v{version}.xlsx",
    sheet_name="results",
    lab=lab,
)
utils.perform_basic_checks(new_df, ref_stn_xls)
new_df = utils.wide_to_long(new_df, lab)

# Add label for data period
new_df["period"] = "new"

# Handle duplicates
dup_csv = os.path.join(
    fold_path, f"{lab.lower()}_{year}_q{qtr}_v{version}_duplicates.csv"
)
new_df = utils.handle_duplicates(new_df, dup_csv, action="drop")

new_df.head()


Checking stations:

The following location IDs have inconsistent names within this template:

The following location names have multiple IDs within this template:

Checking sample dates:
    Done.

Checking for non-numeric data:
    Done.

Checking for values less than zero:
    Done.

Checking for consistent LOD values:
    Done.

Checking NO3 and TOTN:
The following samples have nitrate greater than total nitrogen:
     vannmiljo_code sample_date depth1 depth2  NO3_µg/l  Tot-N_µg/l
866       063-58804  2025-02-03      0      0     300.0         290
1003      067-82874  2025-01-06      0      0     320.0         290
1083      064-82800  2025-01-06      0      0     140.0         120
1101      064-28997  2025-02-24      0      0     110.0         100
1116      064-81557  2025-01-06      0      0      59.0          56
1121      064-81557  2025-02-24      0      0      78.0          70

Checking Al fractions:
The following samples have LAl != RAl - ILAl:
     vannmiljo_code sample_date 

,vannmiljo_code,sample_date,lab,depth1,depth2,par_unit,flag,value,period
0,019-58793,2025-01-07,Eurofins,0.0,0.0,TEMP_°C,,1.2,new
1,019-58793,2025-01-27,Eurofins,0.0,0.0,TEMP_°C,,2.0,new
2,019-58793,2025-02-24,Eurofins,0.0,0.0,TEMP_°C,,2.0,new
3,019-101022,2025-01-07,Eurofins,0.0,0.0,TEMP_°C,,0.6,new
4,019-101022,2025-01-27,Eurofins,0.0,0.0,TEMP_°C,,1.0,new


## 7 . Combine

Combine the `historic` and `new` datasets into a single dataframe in "long" format.

In [11]:
# Combine
df = pd.concat([his_df, new_df], axis="rows")

# Separate par and unit
df[["parameter", "unit"]] = df["par_unit"].str.split("_", n=1, expand=True)
del df["par_unit"]

df.reset_index(drop=True, inplace=True)

df.head()

,vannmiljo_code,sample_date,lab,depth1,depth2,flag,value,period,parameter,unit
0,027-28435,2012-01-02,NIVA (historic),0.0,0.0,=,0.060,historic,ALK,mmol/l
1,027-28435,2012-02-06,NIVA (historic),0.0,0.0,=,0.061,historic,ALK,mmol/l
2,027-28435,2012-03-05,NIVA (historic),0.0,0.0,=,0.050,historic,ALK,mmol/l
3,027-28435,2012-04-02,NIVA (historic),0.0,0.0,=,0.058,historic,ALK,mmol/l
4,027-28435,2012-05-17,NIVA (historic),0.0,0.0,=,0.077,historic,ALK,mmol/l


In [12]:
# VestfoldLAB SiO2 has been fixed in recent Vannmiljø exports
# # Apply correction to historic SIO2
# df["value"] = np.where(
#     (df["lab"] == "VestfoldLAB (historic)") & (df["parameter"] == "SIO2"),
#     df["value"] * 467.5432,
#     df["value"],
# )

# Reclassify (nitrate + nitrite) to nitrate
df["parameter"] = df["parameter"].replace({"N-SNOX": "N-NO3"})

## 8. Check data ranges

A simple method for preliminary quality control is to check whether parameter values are within sensible ranges (as defined in the `parameters_units` table; see Section 4 above). I believe this screening should be implemented differently for the `historic` (i.e. Vannmiljø) and `new` datsets, as follows:

 * For the `historic` data in Vannmiljø, values outside the plausible ranges should be **removed from the dataset entirely**. This is because we intend to use the Vannmiljø data as a reference against which new values will be compared, so it is important the dataset does not contain anything too strange. Ideally, the reference dataset should be carefully manually curated to ensure it is as good as possible, but I'm not sure we have the resouces in this project to thoroughly quality assess the data *already* in Vannmiljø. Dealing with any obvious issues is a good start, though
 
 * For the `new` data, values outside the plausible ranges should be highlighted and checked with the reporting lab
 
**Note:** At present, my code will remove any concentration values of exactly zero from the historic dataset. **Check with Øyvind whether this is too strict**.

In [13]:
# Check ranges
df = utils.check_data_ranges(df)


Checking data ranges for the 'historic' period.
    TEMP: Maximum value of 45.00 is greater than or equal to upper limit (30.00).
    PH: Minimum value of 0.10 is less than or equal to lower limit (1.00).
    KOND: Maximum value of 309.00 is greater than or equal to upper limit (100.00).
    P-TOT: Maximum value of 1200.00 is greater than or equal to upper limit (500.00).
    N-TOT: Maximum value of 5200.00 is greater than or equal to upper limit (4000.00).
    N-NO3: Maximum value of 4900.00 is greater than or equal to upper limit (2000.00).
    LAL: Minimum value of 0.00 is less than or equal to lower limit (0.00).
    CL: Maximum value of 530.00 is greater than or equal to upper limit (100.00).
    SO4: Minimum value of 0.00 is less than or equal to lower limit (0.00).
    SO4: Maximum value of 71.60 is greater than or equal to upper limit (20.00).
    CA: Minimum value of 0.00 is less than or equal to lower limit (0.00).
    CA: Maximum value of 4800.00 is greater than or equal to

In [14]:
# Add to database
df.to_sql(
    name="water_chemistry",
    con=eng,
    if_exists="append",
    index=False,
    method="multi",
    chunksize=1000,
)

317495

In [15]:
eng.close()

## 9. Version 1 summary

Points to note from the initial data exploration:

### 9.1. Data formatting and missing data

No issues identified.
   
### 9.2. Site IDs

No issues identified.
 
### 9.3. Duplicates

There are 34 duplicated samples, of which 16 are marked as "Flomprøve". **There seems to be a genuine duplicate from `024-58895`** (Litleåna oppstrøms doserer) on 03.03.2025.

### 9.4. Range checks

*  The sample collected from `030-58838` (Frafjordelva) on 06.01.2025 may be contaminated. The following values are all very high: conductivity 309 mS/m; chloride 730 mg/l; sulphate 117 mg/l; potasium 19 mg/l; sodium 510 mg/l. 

 * There are 6 samples with NO3 > TOTN, but differences are within lab error according to Eurofins.
    
 * There appear to be some issues with the calculation of LAL. Wherever ILAl is <5 ug/l, it seems LAl has been set to <5 ug/l. However, we have previously agreed that

   > LAl settes til <5 når ILAl <5 og RAl <10. Ved RAl >=10 og ILAl<5 kan LAl beregnes ved å sette ILAl = 5 (gir et konservativt estimat for LAl).

   This has not been done for the following samples:

     * 045-58814 on 2025-02-06
     * 045-58815 on 2025-01-07

  
 * There are two samples where LAl > 20 µg/l and pH > 6.4, which is considered unlikely:

     * 021-86768 on 2025-01-06
     * 022-58904 on 2025-01-30.